In [ ]:
# # download pandas
# pip install pandas

In [ ]:
# # download this to read excel dataset
# pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# pip install spacy

   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
    --------------------------------------- 0.2/12.2 MB 5.1 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/12.2 MB 7.7 MB/s eta 0:00:02
   ---- ----------------------------------- 1.4/12.2 MB 11.5 MB/s eta 0:00:01
   ------- -------------------------------- 2.3/12.2 MB 13.3 MB/s eta 0:00:01
   ------------ --------------------------- 3.7/12.2 MB 17.0 MB/s eta 0:00:01
   ----------------- ---------------------- 5.4/12.2 MB 21.7 MB/s eta 0:00:01
   ------------------------ --------------- 7.5/12.2 MB 23.8 MB/s eta 0:00:01
   ------------------------------- -------- 9.7/12.2 MB 28.3 MB/s eta 0:00:01
   ---------------------------------------  12.2/12.2 MB 43.7 MB/s eta 0:00:01
   ---------------------------------------  12.2/12.2 MB 43.7 MB/s eta 0:00:01
   ---------------------------------------- 12.2/12.2 MB 34.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/183.0 kB ? eta -:--:--


In [ ]:
# pip install spacy textblob ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [1]:
from spacy.cli import download

download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import pandas as pd
import numpy as np
import regex as re
import spacy
from spacy import displacy
from collections import Counter
import requests
from textblob import TextBlob
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

In [3]:
# excel into dataframe
news_df = pd.read_excel("data/news_excerpts_parsed.xlsx")
wiki_df = pd.read_excel("data/sorted_wikileaks_parsed.xlsx")


pd.set_option('display.max_colwidth', None)

In [ ]:
print(news_df.head())

                                                                                                                           Link  \
0                                                  https://edition.cnn.com/2023/09/29/business/starbucks-union-wages/index.html   
1         https://www.channelnewsasia.com/singapore/su-wenqiang-pleads-guilty-billion-dollar-money-laundering-convicted-4234731   
2                                         https://edition.cnn.com/2023/05/22/tech/meta-facebook-data-privacy-eu-fine/index.html   
3  https://www.channelnewsasia.com/singapore/billion-dollar-money-laundering-case-zhang-ruijin-sentenced-15-months-jail-4302416   
4                                 https://edition.cnn.com/2024/03/05/politics/liberty-university-fined-campus-safety/index.html   

                                                                                                                                                                                                                     

In [ ]:
print(wiki_df.head())

  PDF Path  \
0    1.pdf   
1    1.pdf   
2   10.pdf   
3   10.pdf   
4   10.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [ ]:
# # # Use regex to clean the text by removing punctuation??
# # pattern = r'[^\w\s]'
# # news_df['text_cleaned'] = news_df['Text'].apply(lambda x: re.sub(pattern, '', x))

# # Convert to lowercase
# news_df['text_cleaned'] = news_df['Text'].str.lower()
# wiki_df['text_cleaned'] = wiki_df['Text'].str.lower()

# # to-do: handle special chara, missed handling (e.g 1.2 billion, removed puntuation becom 12 billion)

# print(news_df.head())

In [4]:
nlp = spacy.load('en_core_web_sm') # Load spaCy model

In [5]:
news_df['doc'] = news_df['Text'].apply(nlp)
print(news_df['doc'])

0                                                     (Starbucks, violated, federal, labor, law, when, it, increased, wages, and, offered, new, perks, and, benefits, only, to, non, -, union, employees, ,, a, National, Labor, Relations, Board, judge, found, Thursday, ., \n\n, The, decision, is, the, latest, in, a, series, of, NLRB, rulings, finding, that, Starbucks, has, violated, labor, law, in, its, efforts, to, stop, unions, from, forming, in, its, coffee, shops, ., \n\n, “, The, issue, at, the, heart, of, this, case, is, whether, ,, under, current, Board, law, ,, [, Starbucks, ], was, entitled, to, explicitly, reward, employees, ,, ”, for, not, participating, in, union, activity, ,, “, ...)
1                                                                (The, first, suspect, to, plead, guilty, in, Singapore, 's, largest, money, laundering, case, was, convicted, and, sentenced, to, 13, months, ', jail, in, a, district, court, on, Tuesday, (, Apr, 2, ), ., \n\n, Su, Wenqiang, ,, 32, 

In [6]:
wiki_df['doc'] = wiki_df['Text'].apply(nlp)

Tokenization

In [7]:
def extract_entities(doc):
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
    return entities

news_df['entities'] = news_df['doc'].apply(extract_entities)

print(news_df['entities'])

0                                                                                                                                                                                                                                                                                   [(Starbucks, ORG), (National Labor Relations Board, ORG), (Thursday, DATE), (NLRB, ORG), (Starbucks, ORG), (Board, ORG), (Starbucks, PRODUCT), (Mara-Louise Anzalone, ORG)]
1                                                                                                                                                                     [(first, ORDINAL), (Singapore, GPE), (13 months', DATE), (Tuesday, DATE), (2, CARDINAL), (Su Wenqiang, PERSON), (32, DATE), (11, CARDINAL), (More than S$3 billion, MONEY), (US$2.2 billion, MONEY), (Su, PERSON), (10, CARDINAL), (last August, DATE), (Cambodian, NORP), (Fujian, GPE)]
2       [(Meta, ORG), (€1.2 billion, MONEY), ($1.3 billion, MONEY), (European Union, ORG), (EU, ORG), (t

Sentiment Analysis

The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity). The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.

In [8]:
# Process the text using a spaCy pipeline to tokenize,
def extract_sentiment(doc):
    entities_with_sentiment = []

    for ent in doc:
        sentiment = TextBlob(ent.sent.text).sentiment
        entities_with_sentiment.append({
            "text": ent.text,
            'pol': sentiment.polarity,
            'subj': sentiment.subjectivity
        })

    return entities_with_sentiment


news_df['sentiment'] = news_df['doc'].apply(extract_sentiment)

In [ ]:
print(news_df['sentiment'].shape)
print(news_df['sentiment'])

(1509,)
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              [{'text': 'Starbucks', 'pol': 0.06818181818181818, 'subj': 0.7272727272727273}, {'text': 'violated', 'pol': 0.068

In [12]:
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

# Download necessary resources
nltk.download('vader_lexicon')

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

def extract_entity_sentiment_with_context(doc):
    entities_with_sentiment = []

    # Loop through named entities in the document
    for ent in doc.ents:
        # Get surrounding sentence context for better sentiment analysis
        surrounding_sentence = ' '.join([sent.text for sent in doc.sents if ent.text in sent.text])

        # Get sentiment for the entity's surrounding context using both TextBlob and VADER
        textblob_score = TextBlob(surrounding_sentence).sentiment.polarity
        vader_score = analyzer.polarity_scores(surrounding_sentence)['compound']

        entities_with_sentiment.append({
            "entity": ent.text,
            "textblob_pol": textblob_score,
            "vader_compound": vader_score
        })

    return entities_with_sentiment

# Apply function to each document
news_df['entity_sentiments_with_context'] = news_df['doc'].apply(extract_entity_sentiment_with_context)

# Print the first few rows of the result
print(news_df[['doc', 'entity_sentiments_with_context']].head())


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Namyra\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 doc  \
0                                           (Starbucks, violated, federal, labor, law, when, it, increased, wages, and, offered, new, perks, and, benefits, only, to, non, -, union, employees, ,, a, National, Labor, Relations, Board, judge, found, Thursday, ., \n\n, The, decision, is, the, latest, in, a,